# `cudacodec.VideoReader` memory usage example

The memory required by `cudacodec.VideoReader` is mainly influenced by the number of decode surfaces required as demonstrated in this example.  

The minimum number of decode surfaces is determined by the video source and can be increased to increase decoding performance.

Note: A [CUDA context](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#context) which requires several hundred MB of device memory needs to be created before any CUDA functions can be called.  Unfortunately the call to `cv.cuda.DeviceInfo()` will create the cuda context so there is no way to measure how much memory this allocates using the OpenCV API.

This example uses python wheel from https://github.com/cudawarped/opencv-python-cuda-wheels/releases/tag/4.7.0.20221229

In [1]:
import cv2 as cv
import os
import time
b_to_mb = 2**20
vid_root = os.environ['OPENCV_TEST_DATA_PATH'] + "/cv/video/"

## Create `cudacodec.VideoReader` using the default number of decode surfaces

In [2]:
# unfortunately this creates a cuda context therefore mb_free_start is the free memory after context creation
device_info = cv.cuda.DeviceInfo()
mb_free_start = device_info.freeMemory()/b_to_mb

In [3]:
videoReader = cv.cudacodec.createVideoReader(vid_root + "1920x1080.avi")

In [4]:
mb_free_after_creation = device_info.freeMemory()/b_to_mb

In [5]:
# Get number of decode surfaces currently a frame needs to be processed before the format info is valid
videoReader.grab()
format_info = videoReader.format()

In [6]:
mb_used = mb_free_start - mb_free_after_creation
print(f'Total Memory:                            {device_info.totalMemory()/b_to_mb:.2f}MB')
print(f'Free Memory after context creation:      {mb_free_start:.2f}MB')
print(f'Free Memory after creating video reader: {mb_free_after_creation:.2f}MB')
print(f'{mb_used:.2f}MB of internal memory when using {format_info.ulNumDecodeSurfaces} ({format_info.ulWidth}x{format_info.ulHeight}) decode surfaces')

Total Memory:                            8191.50MB
Free Memory after context creation:      6999.00MB
Free Memory after creating video reader: 6973.41MB
25.59MB of internal memory when using 4 (1920x1088) decode surfaces


## Create `cudacodec.VideoReader` using twice as many decode surfaces

In [7]:
params = cv.cudacodec.VideoReaderInitParams()
params.minNumDecodeSurfaces = format_info.ulNumDecodeSurfaces*2

In [8]:
videoReader = cv.cudacodec.createVideoReader(vid_root + "1920x1080.avi",params=params)

In [9]:
mb_used_double_sufaces = mb_free_start - device_info.freeMemory()/b_to_mb

In [10]:
videoReader.grab()
format_info = videoReader.format()
assert format_info.ulNumDecodeSurfaces == params.minNumDecodeSurfaces 

In [11]:
print(f'Memory increase from doubling the number of decode surfaces: {100*(mb_used_double_sufaces - mb_used)/mb_used:.2f}%')

Memory increase from doubling the number of decode surfaces: 62.52%
